### Imports

In [125]:
import pandas as pd
import numpy as np
from typing import List


# Sci-kit learn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split

# PyTorch
import torch
import torch.nn as nn
import torch.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

### Load in Data

In [103]:
df_train = pd.read_csv('../data/spaceship_titanic/train.csv')

In [104]:
# Remove the predicted label
training_labels = df_train.pop('Transported')

In [105]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
dtypes: float64(6), object(7)
memory usage: 883.0+ KB


### Drop Columns

In [106]:
Columns_List = List[str]
def drop_columns(df: pd.DataFrame, cols: Columns_List):
    df.drop(cols, axis = 1, inplace = True)
    return df

df_train = drop_columns(df_train, ["Name", "Cabin"])

### Identify column types

In [107]:
cat_cols = df_train.select_dtypes(include=['object']).columns.to_list()
cont_cols = df_train.select_dtypes(include=['float64']).columns.to_list()

# Exclude passenger ID as categorical column
cat_cols.pop(cat_cols.index('PassengerId'))

print(f"Categorical columns: {cat_cols}")
print(f"Continuous columns: {cont_cols}")

Categorical columns: ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
Continuous columns: ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']


### Encode categorical variables

In [108]:
ord_encoder = OrdinalEncoder()
df_train[cat_cols] = ord_encoder.fit_transform(df_train[cat_cols])

### Feature Engineering

### Impute Missing Values

In [109]:
# Continuous variables

iterative_imputer = IterativeImputer()

df_train[cont_cols] = pd.DataFrame(iterative_imputer.fit_transform(df_train[cont_cols]), columns = cont_cols)

In [110]:
# Categorical variables
categorical_imputer = SimpleImputer(strategy='most_frequent')
df_train[cat_cols] = pd.DataFrame(categorical_imputer.fit_transform(df_train[cat_cols]), columns = cat_cols)


In [111]:
# Sanity check on any remaining missing values
df_train.isnull().sum().sum()

0

#### Get group number

In [112]:
# Group number is in passenger id (first half)
df_train['group'] = df_train['PassengerId'].str.split('_').str[0]
df_train['group'] = pd.to_numeric(df_train['group'])

In [113]:
# Sanity check
df_train['group'][:10]

0    1
1    2
2    3
3    3
4    4
5    5
6    6
7    6
8    7
9    8
Name: group, dtype: int64

In [114]:
# Remove passengerId as its not needed anymore
df_train.drop('PassengerId', axis = 1, inplace=True)

#### Noramlize values

In [115]:
std_scaler = StandardScaler()
normalized_cols = [col + '_norm' for col in cont_cols]
df_train[normalized_cols] = std_scaler.fit_transform(df_train[cont_cols])

In [116]:
# Sanity check on normalized columns
df_train[normalized_cols].head()

,Age_norm,RoomService_norm,FoodCourt_norm,ShoppingMall_norm,Spa_norm,VRDeck_norm
0,0.709373,-0.340420,-0.286919,-0.290836,-0.276256,-0.268140
1,-0.336374,-0.175210,-0.281279,-0.248989,0.211620,-0.229322
2,2.033985,-0.275245,1.954387,-0.290836,5.691115,-0.224911
3,0.291074,-0.340420,0.517218,0.330181,2.682103,-0.097871
4,-0.894105,0.118835,-0.243046,-0.038077,0.225839,-0.266375


#### Compute Feature Importance

In [117]:
df_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,group,Age_norm,RoomService_norm,FoodCourt_norm,ShoppingMall_norm,Spa_norm,VRDeck_norm
0,1.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.709373,-0.340420,-0.286919,-0.290836,-0.276256,-0.268140
1,0.0,0.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,2,-0.336374,-0.175210,-0.281279,-0.248989,0.211620,-0.229322
2,1.0,0.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,3,2.033985,-0.275245,1.954387,-0.290836,5.691115,-0.224911
3,1.0,0.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,3,0.291074,-0.340420,0.517218,0.330181,2.682103,-0.097871
4,0.0,0.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,4,-0.894105,0.118835,-0.243046,-0.038077,0.225839,-0.266375


In [118]:
mi_scores = mutual_info_classif(df_train, training_labels)
mi_scores = pd.Series(mi_scores, name="MI Scores", index=df_train.columns)
mi_scores = mi_scores.sort_values(ascending=False)
mi_scores

CryoSleep            0.109874
RoomService          0.083392
Spa_norm             0.078700
RoomService_norm     0.075452
Spa                  0.071019
VRDeck_norm          0.065167
VRDeck               0.058075
ShoppingMall         0.054294
ShoppingMall_norm    0.052571
FoodCourt            0.048197
FoodCourt_norm       0.037833
HomePlanet           0.022380
group                0.021697
Age                  0.014347
Age_norm             0.012029
VIP                  0.004811
Destination          0.000000
Name: MI Scores, dtype: float64

In [119]:
# Drop Destination and VIP
df_train.drop(['Destination', 'VIP'], axis = 1, inplace = True)

#### Split Data

In [121]:
X_train, X_val, y_train, y_val = train_test_split(df_train, training_labels, test_size=0.2)

In [126]:
class SSTitanic(Dataset):
    
    def __init__(self, csv_path:str):
        """Initialize a Dataset Class for SS Titanic

        Args:
            csv_path (str): path to the csv file
        """
        # Init the dataset
        df = pd.read_csv(csv_path)
        
        # Split the data into X and y
        self.X = ""
        self.y = ""
        
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # Convert idx from tensor to list due to pandas bug (that arises when using pytorch's random_split)
        if isinstance(idx, torch.Tensor):
            idx = idx.tolist()
            
        return [self.X.iloc[idx].values, self.y[idx]]

#### Model

In [129]:
# Define the model

class Net(nn.Module):
    
    def __init__(self, d_in, h = 15, d_out = 1):
        super.__init__()
        
        self.fc1 = nn.Linear(d_in, h)
        self.fc2 = nn.Linear(h, d_out)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        return x.squeeze()

#### Training Loop

In [128]:
def train(csv_file, n_epochs = 100):
    """Train the model

    Args:
        csv_file (_type_): _description_
        epochs (int, optional): _description_. Defaults to 100.
    """
    
    # Load the dataset
    dataset = SSTitanic(csv_file)
    
    # Split into train and val
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    trainset, valset = random_split(dataset, [train_size, val_size])
    
    # DataLoaders
    trainloader = DataLoader(trainset, batch_size = 200, shuffle = True)
    valloader = DataLoader(valset, batch_size = 200, shuffle = False)
    
    # Set device
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # Set the model settings
    D_in, H = 19, 15
    net = Net(D_in, H).to(device)
    
    # Loss function
    criterion = nn.MSELoss()
    
    # Optimizer
    optimizer = torch.optim.Adam(net.parameters(), weight_decay = 0.0001)
    
    # Train the network
    loss_per_iter = list()
    loss_per_batch = list()
    
    for epoch in range(n_epochs):
        
        # Define params each epoch
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(trainloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Perform the usual actions of zero-ing gradients
            optimizer.zero_grad()
            
            # Forward, backward, step
            outputs = net(inputs.float())
            loss = criterion(outputs, labels.float())
            loss.backward() # Backward
            optimizer.step() # Step
            
            # Save loss to plot
            running_loss += loss.item()
            loss_per_iter.append(loss.item())
        
        loss_per_batch.append(running_loss / (i+1))
        running_loss = 0.0